In [8]:
import pandas as pd
AsStats=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aviasales detailed report.csv',delimiter=';')
PartnerStats=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/partner details report.csv',delimiter=';')
PartnerRaw=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/partner raw stats data.csv',delimiter=';')
#подключаем pandas и загружаем файлы, предварительно преобразованные в csv. 
#Также предварительно формат ячеек в столбцах с номерами заказов и ценой был изменён на числовой.

In [4]:
PartnerStats.dropna(axis = 0, how ='all', inplace = True)
#удаляем строки и столбцы с пустыми значениями из всех таблиц

In [5]:
PartnerStats.dropna(axis = 1, how ='all', inplace = True)

In [ ]:
PartnerRaw.dropna(axis = 0, how ='all', inplace = True)

In [ ]:
PartnerRaw.dropna(axis = 1, how ='all', inplace = True)

In [ ]:
AsStats.dropna(axis = 0, how ='all', inplace = True)

In [ ]:
AsStats.dropna(axis = 1, how ='all', inplace = True)

In [ ]:
PartnerStats.rename({'OrderID': 'orderid'}, axis=1, inplace=True)

In [ ]:
AsStats.rename({'OrderNumber': 'orderid'}, axis=1, inplace=True)
#приводим признак номер заказа к единому виду для последующего слияния таблиц

In [ ]:
merged=pd.merge(AsStats, PartnerStats, on='orderid', how='outer', indicator=True)
#объединяем таблицы с финальной статистикой AS и партнёра по номерам заказов

In [ ]:
merged['_merge'].value_counts()
#проверяем число совпадений по номерам заказов и видим, 
#что в нашей стате оказались 122 букинга, которых нет в стате партнёра, 
#а он передал нам 166 букингов, которых не было в нашей стате — это и есть расхождение.
#из визуальных наблюдений можно добавить, что в файле от партнёра есть букинги с нулевой стоимостью, 
#а также дубликаты букингов с разными признаками (DOM и INT) и разной стоимостью (в том числе отрицательной).

both          680
right_only    166
left_only     122
Name: _merge, dtype: int64

In [ ]:
#вторая часть задания
PartnerRaw1=PartnerRaw.drop_duplicates(subset=['orderid'])
#удаляем дубликаты по номерам заказов из таблицы с сырыми данными партнёра 

In [ ]:
AsStats['Profit'].sum()
#рассчитываем общую комиссию по нашей стате (просто чтобы показать, что она отличается от вводных в тестовом)

576937.0

In [ ]:
final=pd.merge(AsStats, PartnerRaw1, on="orderid", how='left')
#объединяем таблицы, чтобы получить признак доместика/международки
#сливаем данные по левой таблице, т.к. задание – рассчитать профит по броням именно из нашей статы. 

In [ ]:
final1=pd.merge(final, PartnerStats, on="orderid", how='left')
#объединяем таблицы, чтобы получить стоимость бронирований партнёра

In [ ]:
domestic=final1[final1['type']=='DOM']
#делим датафрейм на букинги доместик и интернешнл.

In [ ]:
international=final1[final1['type']=='INT']

In [ ]:
(international['Value'].sum() * 0.015) + (len(domestic.index) * 40)
#рассчитываем общую комиссию в INR как сумму 1.5% от суммы стоимостей INT букингов и 
#40*число записей с признаком DOM

125104.7

In [ ]:
#Можно посчитать в рублях по курсу из вводного, где 164335 INR = 146186 RUB:
((international['Value'].sum() * 0.015) + (len(domestic.index) * 40)) * 146186 / 164335

111288.2567572337

In [ ]:
#Вывод: судя по тому, что в итоге получилась какая-то хуета, 
#нужно делать больше манипуляций с данными и их сравнений, например, по статусу заказов,
#а также рассматривать список букингов, которые не совпали с партнёром, искать закономерности,
#делать гипотезы и проверять их.